## Setup

In [9]:
from dotenv import load_dotenv
from agentic.logging_config import configure_logging
from utils import chat_interface

In [2]:
# Import the compiled Supervisor graph and optional chat interface
from agentic.workflow import supervisor

## Run

In [3]:
# Helper to invoke supervisor with a session (thread_id)
def run(message: str, thread_id: str = "nb-thread-1"):
    return supervisor.invoke(
        {"messages": [("user", message)]},
        config={"configurable": {"thread_id": thread_id}},
    )

In [4]:
# Account-ops route example
res_account = run("I need help with my subscription status", "nb-account")
print(res_account)

{'messages': [HumanMessage(content='I need help with my subscription status', additional_kwargs={}, response_metadata={}, id='9536998d-2ae4-44a5-bbdd-b97631bc8b3f'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 132, 'total_tokens': 144, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CaOcTxC68JRQzCSBI8GNKVGQUwZQq', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='lc_run--7e71d8a7-a34a-43d8-aea5-4ef5abd30aa4-0', tool_calls=[{'name': 'transfer_to_account_ops', 'args': {}, 'id': 'call_IjIlVxNuPyl6gQpwTGJJBnAL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 132, 'ou

In [5]:
# Knowledge route + escalation-style examples
res_kb = run("How do I reset my password?", "nb-knowledge")
print(res_kb)

res_esc = run("I was charged twice and need a refund", "nb-escalation")
print(res_esc)

{'messages': [HumanMessage(content='How do I reset my password?', additional_kwargs={}, response_metadata={}, id='01178f30-7691-4243-ab95-3b3fb875956f'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 132, 'total_tokens': 144, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CaOcwdcSVW4PWA7MEblLOzYbuW60R', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='lc_run--ea21046a-79f7-453e-a10c-b9dfbc86af0d-0', tool_calls=[{'name': 'transfer_to_account_ops', 'args': {}, 'id': 'call_eiDJJ0NisJFaO7deIUjyOZAw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 132, 'output_tokens'

In [7]:
# Inspect session memory (short-term) for a specific thread
history = list(
    supervisor.get_state_history(
        config={"configurable": {"thread_id": "nb-account"}}
    )
)
# Latest checkpoint’s messages for that session
history[0].values["messages"] if history else []

[HumanMessage(content='I need help with my subscription status', additional_kwargs={}, response_metadata={}, id='9536998d-2ae4-44a5-bbdd-b97631bc8b3f'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 132, 'total_tokens': 144, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CaOcTxC68JRQzCSBI8GNKVGQUwZQq', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='lc_run--7e71d8a7-a34a-43d8-aea5-4ef5abd30aa4-0', tool_calls=[{'name': 'transfer_to_account_ops', 'args': {}, 'id': 'call_IjIlVxNuPyl6gQpwTGJJBnAL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 132, 'output_tokens'

In [10]:
# Interactive chat (press q to exit)
# Uses the same compiled supervisor and persists via thread_id
chat_interface(supervisor, "nb-chat")

User: Hi
Assistant: Hello! How can I assist you today?
User: My card was stolen
Assistant: It seems the account operations agent needs more details to assist you further. Let me know how you would like to proceed or if you need any additional support!
User: my card was stolen what should i do
Assistant: If your card was stolen, here are the immediate steps you should take:

1. **Contact Your Bank or Card Issuer**: Report the theft immediately to freeze your account and prevent unauthorized charges.

2. **Monitor for Unauthorized Transactions**: Keep an eye on your account for any transactions you don't recognize.

3. **Update Automatic Payments**: If you have any services linked to the stolen card, update them with your new card information once you receive it.

4. **File a Police Report**: Depending on the situation and your local laws, it might be a good idea to file a police report.

5. **Consider Identity Theft Protection**: If you're concerned about identity theft, you may want to

In [11]:
# Persistent memory demo: store a resolution and read it back from DB
import json
from agentic.tools.memory_tools import get_user_ticket_history, get_ticket_details
from agentic.memory_manager import MemoryManager

demo_user_id = "a4ab87"  # use an ID that exists in your DB setup

hist = json.loads(get_user_ticket_history.invoke({"external_user_id": demo_user_id, "limit": 3}))
if hist.get("found") and hist.get("tickets"):
    ticket_id = hist["tickets"][0]["ticket_id"]

    before = json.loads(get_ticket_details.invoke({"ticket_id": ticket_id}))
    print("Before:", before.get("status"), "messages:", before.get("message_count"))

    ok = MemoryManager().store_interaction(
        ticket_id=ticket_id,
        user_id=demo_user_id,
        category="account",
        resolution="Notebook demo: confirmed subscription details and resolved.",
        status="resolved",
    )
    print("Store OK:", ok)

    after = json.loads(get_ticket_details.invoke({"ticket_id": ticket_id}))
    print("After:", after.get("status"), "messages:", after.get("message_count"))
else:
    print("No demo user found. Run the DB setup notebooks first.")

Before: resolved messages: 2
Store OK: True
After: resolved messages: 3
